# 08 - Création du dataset de données traitées pour entrainement <br> avec filtre des évènements par id
(sans encodage, ni normalisation)

Ce notebook génère 1 csv :

- dataset_for_training.csv : analyse et nettoyage des variables explicatives

Etapes : 




## A) Imports

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
# from ydata_profiling import ProfileReport # pip install ydata_profiling ipywidgets
import matplotlib.pyplot as plt

In [2]:
# source path to datasets
path = '../data/'
data = 'dataset_for_preprocess_id_events_filtered_07.csv'
save_csv = '../data/dataset_for_training_id_events_filtered_08.csv'

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(os.path.join(path, data), header=0, parse_dates=True, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 40 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   started_at                                            19 non-null     object 
 1   ended_at                                              19 non-null     object 
 2   paperHeight_job                                       19 non-null     int64  
 3   paperWidth_job                                        19 non-null     int64  
 4   scanner_mode                                          19 non-null     int64  
 5   bars_job                                              19 non-null     int64  
 6   copies_per_run                                        19 non-null     int64  
 7   jobId                                                 19 non-null     int64  
 8   total_copies_requested                                19 non-n

In [4]:
# # Liste des colonnes à vérifier
# colonnes_a_verifier = ['topMargin_remoteScannerRegistration', 'leftMargin_remoteScannerRegistration']


# # Afficher la répartition des valeurs pour chaque colonne dans la liste
# for col in colonnes_a_verifier:
#     print(f"Valeurs uniques pour {col}: {df[col].unique()}")
#     print(f"Répartition des valeurs pour {col}:")
#     print(df[col].value_counts())
#     print("\n") 

## B) Analyse des variables

In [5]:
# remise à zero des index
df.reset_index(drop=True,inplace=True)

In [6]:
# suppression des colonnes de metrics
# metrics_cols = [
#     'status', 
#     'source_events', 
#     'timestamp', 
#     'criticality_events', 
#     'name_modules', 
#     'type_modules',
#     'generation_modules', 
#     'value_counters_modules', 
#     'name_counters_modules',
#     'name_connected_operators',
#     'level_connected_operators',
#     'varnishLevelsTargetvolume',
#     'varnishLevelsTotalvolume'
#     ]
metrics_cols = [
    'timestamp', 
    'criticality_events', 
    'index',
    ]
df.drop(columns=metrics_cols, axis=0, inplace=True)

In [7]:
df.columns

Index(['started_at', 'ended_at', 'paperHeight_job', 'paperWidth_job',
       'scanner_mode', 'bars_job', 'copies_per_run', 'jobId',
       'total_copies_requested', 'LED', 'bars_event', 'drops', 'dithering',
       'deadPixelsOffset', 'operator_level', 'operator', 'speed_ifoil',
       'ifoil', 'optifoil_ifoil', 'stampAreas_ifoil', 'heater1Enabled_ifoil',
       'heater1Temperature_ifoil', 'x_imageLayout', 'y_imageLayout',
       'paperWidth_event', 'paperHeight_event', 'speed', 'power_irDryers',
       'power_uvDryers', 'leftMargin_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'mode_remoteScannerRegistration', 'jobState', 'total_copies',
       'varnishConsumptionVarnish_3d', 'identification_events'],
      dtype='object')

In [8]:
# suppression des colonnes en doublon entre job_events et jobs
job_events_cols = [
    'bars_event',
    'paperWidth_event', 
    'paperHeight_event',
    ]
df.drop(columns=job_events_cols, axis=0, inplace=True)

In [9]:
# suppression des colonnes de variables catégorielles
cat_cols = [
    'operator_level', 
    'operator',
    'jobState',
    'jobId'
   ]
df.drop(columns=cat_cols, axis=0, inplace=True)

In [10]:
# suppression des colonnes datetime et ajout d'une colonne duration
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
# Calculer la différence de temps entre les deux colonnes
df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df.drop(columns=['started_at', 'ended_at'], axis=0, inplace=True)

In [11]:
df.head(3)

,paperHeight_job,paperWidth_job,scanner_mode,bars_job,copies_per_run,total_copies_requested,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,leftMargin_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies,varnishConsumptionVarnish_3d,identification_events,duration
0,520,740,1,2,0,350,10,1,False,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.000000,387,74.200
1,520,740,1,2,39,280,10,1,False,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,360,45,100,0,10,10,1,39,18.167730,354,334.853
2,1020,700,1,2,149,300,10,1,False,0,24.6,False,True,"[{'id': 1, 'end': 1020, 'start': 0, 'height': ...",False,115,354,0,300,45,100,0,10,12,1,149,300.958011,355,1512.034


### a) Profiling report

In [12]:
# profile = ProfileReport(df, title="Profiling Report")
# profile.to_notebook_iframe()

## C) Nettoyage

In [13]:
# Liste des colonnes pour lesquelles on veut voir les valeurs uniques
colonnes_a_verifier = ['dithering','deadPixelsOffset']

# Afficher les valeurs uniques pour chaque colonne dans la liste
for col in colonnes_a_verifier:
    print(f"Valeurs uniques pour {col}: {df[col].unique()}")


Valeurs uniques pour dithering: [False]
Valeurs uniques pour deadPixelsOffset: [0]


In [14]:
# supression des colonnes dont les valeurs sont constantes
contstant_cols = ['dithering','deadPixelsOffset']
df.drop(columns=contstant_cols, axis=0, inplace=True)

In [15]:
df.head(5)

,paperHeight_job,paperWidth_job,scanner_mode,bars_job,copies_per_run,total_copies_requested,LED,drops,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,leftMargin_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies,varnishConsumptionVarnish_3d,identification_events,duration
0,520,740,1,2,0,350,10,1,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.000000,387,74.200
1,520,740,1,2,39,280,10,1,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,360,45,100,0,10,10,1,39,18.167730,354,334.853
2,1020,700,1,2,149,300,10,1,24.6,False,True,"[{'id': 1, 'end': 1020, 'start': 0, 'height': ...",False,115,354,0,300,45,100,0,10,12,1,149,300.958011,355,1512.034
3,1020,700,1,2,2,300,10,1,24.6,False,True,"[{'id': 1, 'end': 1020, 'start': 0, 'height': ...",False,115,354,0,300,45,100,0,10,12,1,2,4.039998,355,71.227
4,1020,700,1,2,22,300,10,1,24.6,False,True,"[{'id': 1, 'end': 1020, 'start': 0, 'height': ...",False,115,354,0,300,45,100,0,10,12,1,22,44.443352,357,413.095


Données disponibles au démarrage d'un job :
- 1  thumbnail* : chemin vignette projet
- 2  total_copies : nombre de copies imprimées
- 3  started_at : date de début
- 4  ended_at : date de fin
- 5  machineId* : identifiant machine
- 6  speed : vitesse d'impression
- 7  operator : opérateur
- 8  operator_level : niveau de l'opérateur
- 9  first_page_image_path_on_machine* : chemin de l'image
- 10 paperHeight : hauteur du substrat
- 11 paperWidth : largeur du substrat
- 12 paperName : appelation du substrat
- 13 paperThickness* : épaisseur du substrat (valeur unique à 0)
- 14 id_on_machine : identifiant unique du travail d'impression
- 15 total_copies_requested : nombre de copies demandées
- 16 job_thumbnail_id* : identifiant de vignette image
- 17 uses_ifoil : impression utilisant de la dorure
- 18 uses_iper* : impression nécéssitant l'iper (valeur unique à True)
- 19 scanner_mode : niveau de config du scanner
- 20 iper_bvar_count : compteur # TODO vérifier la fusion notebook 06
- 21 varnishConsumptionVarnish_3d : consommation de vernis en 3d
- 22 varnishConsumptionVarnish_2d* : consommation de vernis en 2d

*\*variables déjà supprimées dans le notebook 05 ou 06*

In [16]:
df.columns

Index(['paperHeight_job', 'paperWidth_job', 'scanner_mode', 'bars_job',
       'copies_per_run', 'total_copies_requested', 'LED', 'drops',
       'speed_ifoil', 'ifoil', 'optifoil_ifoil', 'stampAreas_ifoil',
       'heater1Enabled_ifoil', 'heater1Temperature_ifoil', 'x_imageLayout',
       'y_imageLayout', 'speed', 'power_irDryers', 'power_uvDryers',
       'leftMargin_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'mode_remoteScannerRegistration', 'total_copies',
       'varnishConsumptionVarnish_3d', 'identification_events', 'duration'],
      dtype='object')

In [17]:
columns_to_add = ['redScore_gridMode_remoteScannerRegistration',
                  'redScore_cropmarksMode_remoteScannerRegistration',
                  'redScore_fullScannerMode_remoteScannerRegistration']

df[columns_to_add] = 2561


In [19]:
# colonnes à conserver dispos dans jobs
jobs_cols_to_keep = [
    'duration',
    'total_copies',
    'speed',
    'paperHeight_job', 
    'paperWidth_job', 
    'total_copies_requested',
    'ifoil',
    'scanner_mode', 
    'bars_job', # TODO vérifier les valeurs de la variable d'origine iper_bvar_count
    'varnishConsumptionVarnish_3d'
    ]
# colonnes à conserver dispos dans job events avec le tag start
jobevents_cols_to_keep = [
    'LED',
    'drops', 
    'speed_ifoil',
    'optifoil_ifoil', 
    'stampAreas_ifoil',
    'heater1Enabled_ifoil', 
    'heater1Temperature_ifoil',
    'x_imageLayout',
    'y_imageLayout', 
    'power_irDryers', 
    'power_uvDryers',
    'leftMargin_remoteScannerRegistration',
    'redScore_gridMode_remoteScannerRegistration',
    'redScore_cropmarksMode_remoteScannerRegistration',
    'redScore_fullScannerMode_remoteScannerRegistration',
    'blueScore_fullScannerMode_remoteScannerRegistration',
    'greenScore_fullScannerMode_remoteScannerRegistration',
    'mode_remoteScannerRegistration'
    ]
# colonnes à conserver pour la prédiction
metrics_cols_to_keep = ['identification_events']

# fusion des listes de colonnes à conserver
cols_to_keep = jobs_cols_to_keep + jobevents_cols_to_keep + metrics_cols_to_keep

# dataframe des variables conservées
df_to_encode = df[cols_to_keep].copy()

In [20]:
df_to_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   duration                                              19 non-null     float64
 1   total_copies                                          19 non-null     int64  
 2   speed                                                 19 non-null     int64  
 3   paperHeight_job                                       19 non-null     int64  
 4   paperWidth_job                                        19 non-null     int64  
 5   total_copies_requested                                19 non-null     int64  
 6   ifoil                                                 19 non-null     bool   
 7   scanner_mode                                          19 non-null     int64  
 8   bars_job                                              19 non-n

In [21]:
df_to_encode.drop(columns=['stampAreas_ifoil'], axis=0, inplace=True)

In [ ]:
# cols = df_to_encode.columns
# for col in cols:
#     if df_to_encode[col].nunique() > 50 :
#         print(col, df_to_encode[col].nunique(), 'min', df_to_encode[col].min(), 'max', df_to_encode[col].max())
#     else :
#         print(col, df_to_encode[col].unique())

In [22]:
df_to_encode.identification_events.unique()

array([387, 354, 355, 357, 356, 454], dtype=int64)

## D) Output

In [23]:
df_to_encode.to_csv(save_csv)